# Project Steps
 ## Main Steps:

* Camera calibration and distortion correction.
* Color/gradient threshold.
* Perspective transform.
* Detect lane lines.

## Extra Step:
* Determine the lane curvature.

In [0]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Camera Calibration  and Distortion Correction


*   Utils


In [0]:
def cameraCalibrate(image):
    # Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
    import glob

    #read in and make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')

    #Arrays to store objects points and image points from all the images
    objpoints = [] #3d points
    imgpoints = [] #2d points

    #prepare points
    objp  = np.zeros((9*6,3),np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)


    for fname in images:
        #read in each image
        img = mpimg.imread(fname)
        #convert image to gray scale  
        gray  = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        #find corners  
        ret,corners = cv2.findChessboardCorners(gray,(9,6),None)
        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)
            
    shape =(img.shape[1],img.shape[0])
    ret, mtx, dist, rvect, tvect = cv2.calibrateCamera(objpoints,imgpoints,shape,None,None)
    
    #undistored the image
    undistorted_image = cv2.undistort(image, mtx, dist, None, mtx)
    return undistorted_image

def perspective_transform(undistorted):   
    img_size = (undistorted.shape[1], undistorted.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    transformed_img = cv2.warpPerspective(undistorted, M, img_size, flags=cv2.INTER_LINEAR)
    return transformed_img

**Test Calibration**

In [0]:
task_image = mpimg.imread('./test3.jpg')

f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Original image', fontsize=20)
ax1.imshow(task_image)
ax2.set_title('calibrated image', fontsize=20)
task_image=cameraCalibrate(task_image)
ax2.imshow(cameraCalibrate(task_image))

# make sure that the points follow the right arrangement whether it's clockwise or counter-clockwise
# source and destination points must have the same arrangement whether it's clockwise or counter-clockwise
# The points in src array are (x,y).
# task_image = cameraCalibrate(task_image)

w = task_image.shape[1]
h = task_image.shape[0]
src = np.float32([[w/8,h],[3*w/8,4*h/6],
                  [5*w/8,4*h/6],[7*w/8,h]])

# visualize your mask region
f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
ax1.set_title('source area', fontsize=20)
ax1.imshow(task_image)
ordered_y = [src[0][1],src[1][1],src[2][1],src[3][1],src[0][1]]
ordered_x = [src[0][0],src[1][0],src[2][0],src[3][0],src[0][0]]
ax1.plot(ordered_x,ordered_y , color='red', alpha=0.7,
    linewidth=3, solid_capstyle='round', zorder=2)

# make sure that the points follow the right arrangement whether it's clockwise or counter-clockwise
# source and destination points must have the same arrangement whether it's clockwise or counter-clockwise
# The points in src array are (x,y).
dst = np.float32([[2*w/10,h],[2*w/10,2*h/10],
                  [8*w/10,2*h/10],[8*w/10,h]])
# dst = np.float32([[0,8*h/10], [0,0],
#                  [w,0], [w,h]])

# visualize your mask region
f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
ax1.set_title('destination area', fontsize=20)
ax1.imshow(task_image)
ordered_y = [dst[0][1],dst[1][1],dst[2][1],dst[3][1],dst[0][1]]
ordered_x = [dst[0][0],dst[1][0],dst[2][0],dst[3][0],dst[0][0]]
ax1.plot(ordered_x,ordered_y , color='red', alpha=0.7,
    linewidth=3, solid_capstyle='round', zorder=2)

# Undistord the image then apply perspective transformation
transformed_image = perspective_transform(task_image)
# visualize your results
f, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Original image', fontsize=20)
ax1.imshow(task_image)
ax2.set_title('Transformed image', fontsize=20)
ax2.imshow(transformed_image)

# Gradient / Color Threshold

# Perspective Transform

# Detect Lane Lines

After finishing the previous steps You now have a thresholded warped image and you're ready to map out the lane lines! There are many ways you could go about this, but here's one example of how you might do it:
### Peaks in a Histogram and Sliding Windows
* After applying calibration, thresholding, and a perspective transform to a road image, you should have a binary image where the lane lines stand out clearly. However, you still need to decide explicitly which pixels are part of the lines and which belong to the left line and which belong to the right line.
* we can use the two highest peaks from our histogram as a starting point for determining where the lane lines are, and then use sliding windows moving upward in the image (further along the road) to determine where the lane lines go.
#### steps:
  1. split the histogram into two sides, one for each lane line.
  2. Set up sliding windows and window hyperparameters:
     * set a few hyperparameters related to our sliding windows, and set them up to iterate across the binary activations in the image. These hyperparameters are:
        1. **W_Number**; number of sliding windows.
        2. **Margin**; the width of each window.
        3. **Minimum_pixels**; used as a threshold to recenter the next sliding window.
        4. **Window_Height**; computed from number of pixels and image height.
  3. Loop through each window in W_Number.
  4. Find the boundaries of our current window. This is based on a combination of the current window's starting point      , as well as the margin you set in the hyperparameters.
  5. Use cv2.rectangle to draw these window boundaries onto visualization image.
  6. Now that we know the boundaries of our window, find out which activated(non zero) pixels actually fall into the window.
  7. Append these non zero pixels to two different arrays one for the right line and the other for the left line.
  8. If the number of pixels you found in Step **6** are greater than your hyperparameter Minimum_pixels, re-center our window based on the mean position of these pixels.
  9. Now that we have found all our pixels belonging to each line through the sliding window method, it's time to fit a polynomial to the line.

## This function is to draw back the lane on the original image
**N.B.** Add an inverse option in the perspective function so that we can transform the image back to it's original perspective. This can be implemented by swapping **src** and **dst** in cv2.getPerspectiveTransform(src, dst) when a certain flag is set to 1.

In [0]:
#draw lane
# left_fit and right_fit generated from step 9 in lane Detection
# bird_eye the image after thresholding and perspective transform
# read the inverse note below in the code
def draw_lane(img,bird_eye,left_fit,right_fit):
    tmp_image     = np.copy(img)
    if right_fit is None or left_fit is None:
        return img
    
    zero          = np.zeros_like(bird_eye).astype(np.uint8)
    layered_image = np.dstack((zero,zero,zero))
    
    ploty      = np.linspace(0, bird_eye.shape[0]-1, bird_eye.shape[0] )
    left_fitx  = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2] 
    
    #formatting the points
    left   = np.array([np.transpose(np.vstack([right_fitx,ploty]))])
    right  = np.array([np.flipud(np.transpose(np.vstack([left_fitx,ploty])))])
    points = np.hstack((left,right))
    
    #form lane
    cv2.fillPoly(layered_image,np.int_([points]),(0,255,0))
    cv2.polylines(layered_image,np.int32([right]),isClosed = False,color=(255,0,0),thickness = 20)
    cv2.polylines(layered_image,np.int32([left]),isClosed = False,color=(255,0,0),thickness = 20)
    
    # The inverse perspective transfom note
    # use the inverse perspective option mentioned in the note above to transform back the layered_image
    inversed = perspective_transform(layered_image,1)
    
    output   = cv2.addWeighted(tmp_image,1,inversed,0.5,0)
    return output

## Determine The Lane Curvature
You're getting very close to a final result! You have a thresholded image, where you've estimated which pixels belong to the left and right lane lines, and you've fit a polynomial to those pixel positions. Next we'll compute the radius of curvature of the fit.

## Curvature in Pixels
In the last step we computed the lane line pixels using their x and y pixel positions to fit a second order polynomial curve: $$f(y) = Ay^2+By+C $$
in this step you will compute the radius of curvature at the closest point to the vehicle.

**Radius of Curvature Equation:**
$$R\_Curve = \frac{[1+(\frac{dx}{dy})^2]^{3/2}}{|\frac{d^2x}{dy^2}|}$$

$$f'(y) = \frac{dx}{dy} = 2Ay+B$$

$$f''(y) = \frac{d^2x}{dy^2} =A$$

## From Pixels to Real World
* Great! You've now calculated the radius of curvature for our lane lines. But now we need to stop and think... We've calculated the radius of curvature based on pixel values, so the radius we are reporting is in pixel space, which is not the same as real world space. So we actually need to repeat this calculation after converting our x and y values to real world space.

* This involves measuring how long and wide the section of lane is that we're projecting in our warped image. We could do this in detail by measuring out the physical lane in the field of view of the camera, but for this project, you can assume that if you're projecting a section of lane similar to the images above, the lane is about 30 meters long and 3.7 meters wide.